In [1]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

from nltk.tag.stanford import StanfordPOSTagger
from stanford_postagger.stanford_wrapper import StanfordPOSTagger as StanfordPOSTaggerWrapper

from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

import scipy
from sklearn.grid_search import RandomizedSearchCV

/home/agus/anaconda3/envs/basic/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/agus/anaconda3/envs/basic/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
%load_ext autoreload
%autoreload 2

# Read Dataset

In [3]:
f = open('datasets/conll2003/train.txt', 'r')
lines = f.readlines()
f.close()

del lines[0]
del lines[0]

dataset = []
sentence = []
for line in lines:
    splitter = line.strip().split(' ')
    if splitter[0] == '':
        continue
    elif (splitter[0] == '-DOCSTART-'):
        dataset.append(sentence)
        sentence = []
    else:
        token = splitter[0]
        tag = splitter[3]
        sentence.append((token, tag))

In [4]:
def convert_conlltxt2dataset(filename):
    f = open(filename, 'r')
    lines = f.readlines()
    f.close()
    
    del lines[0]
    del lines[0]
    
    dataset = []
    sentence = []
    for line in lines:
        splitter = line.strip().split(' ')
        if splitter[0] == '':
            continue
        elif (splitter[0] == '-DOCSTART-'):
            dataset.append(sentence)
            sentence = []
        else:
            token = splitter[0]
            tag = splitter[3]
            sentence.append((token, tag))
    return dataset

In [5]:
validation_dataset = convert_conlltxt2dataset('datasets/conll2003/valid.txt')
test_dataset = convert_conlltxt2dataset('datasets/conll2003/test.txt')

# Add Postag to Dataset

## Example

In [6]:
postagger = StanfordPOSTaggerWrapper()
postag = postagger.tag('+44 171')
postag

[('+44', 'CD'), ('171', 'CD')]

In [7]:
def add_postag2dataset(dataset):
    postagger = StanfordPOSTaggerWrapper()
    dataset_with_postag = []
    for sent in dataset:
        postagged_sent = []
        for index, (token, tag) in enumerate(sent):
            postagged_token = postagger.tag(token)
            postagged_sent.append((token, postagged_token[0][1], tag))
        dataset_with_postag.append(postagged_sent)
        
    return dataset_with_postag

postagged_validation_dataset = add_postag2dataset(validation_dataset)
postagged_test_dataset = add_postag2dataset(test_dataset)

# Delete Unused Dataset
del validation_dataset
del test_dataset

# Extract Feature

In [8]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    # Ortographic Feature, Word, POSTag & N-Gram
    features = {
        'word': word,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[:2]': word[:2],
        'word[:3]': word[:3],
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.isupper()': word.isupper(),
        'postag': postag,
        'postag[:2]': postag[:2]
    }
    
    # Position
    features.update({
        'pos_front': i,
        'pos_end': len(sent) - i
    })
    
    # Bag Of Words
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent) - 1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2postag(sent):
    return [postag for token, postag, label in sent]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

# Feature Extraction

In [9]:
X_val = [sent2features(sent) for sent in postagged_validation_dataset]
y_val = [sent2labels(sent) for sent in postagged_validation_dataset]

X_test = [sent2features(sent) for sent in postagged_test_dataset]
y_test = [sent2labels(sent) for sent in postagged_test_dataset]

del postagged_validation_dataset
del postagged_test_dataset

# Load Word Embedding and Add to The Feature

In [10]:
def load_word_embedding_model(filename):
    f = open(filename, 'r', encoding='utf-8')
    lines = f.readlines()
    del lines[0]

    embedding_dict = {}
    counter = 0
    for line in lines:
        line = line.strip()
        arr = line.split(' ')
        word = arr[0]
        vectors = [float(x) for x in arr[1:]]
        embedding_dict[word] = vectors
    return embedding_dict

def add_word_embedding2feature(embedding_dict, feature):
    VECTOR_SPACE_SIZE = 50
    for i, sentence in enumerate(feature):
        for j, token in enumerate(sentence):
            word = token['word']
            vector = []
            if word in embedding_dict:
                vector = embedding_dict[word]
            else:
                vector = [0 for i in range(VECTOR_SPACE_SIZE)]
            for k in range(len(vector)):
                wk = 'w{}'.format(k+1)
                feature[i][j][wk] = vector[k]
    return feature

from copy import deepcopy

In [11]:
cbow_embedding_dict = load_word_embedding_model('models/word_embedding/cbow_model_5.txt')
X_cbow = deepcopy(X_test)
X_cbow = add_word_embedding2feature(cbow_embedding_dict, X_cbow)

X_val_cbow = deepcopy(X_val)
X_val_cbow = add_word_embedding2feature(cbow_embedding_dict, X_val_cbow)

del cbow_embedding_dict

In [12]:
sg_embedding_dict = load_word_embedding_model('models/word_embedding/sg_model_5.txt')
X_sg = deepcopy(X_test)
X_sg = add_word_embedding2feature(sg_embedding_dict, X_sg)

X_val_sg = deepcopy(X_val)
X_val_sg = add_word_embedding2feature(sg_embedding_dict, X_val_sg)

del sg_embedding_dict

In [13]:
fasttext_embedding_dict = load_word_embedding_model('models/word_embedding/fasttext_model_5.txt')
X_fasttext = deepcopy(X_test)
X_fasttext = add_word_embedding2feature(fasttext_embedding_dict, X_fasttext)

X_val_fasttext = deepcopy(X_val)
X_val_fasttext = add_word_embedding2feature(fasttext_embedding_dict, X_val_fasttext)

del fasttext_embedding_dict

In [14]:
del X_test

In [15]:
X_cbow[0][0]

{'word': 'SOCCER',
 'word.lower()': 'soccer',
 'word[-3:]': 'CER',
 'word[-2:]': 'ER',
 'word[:2]': 'SO',
 'word[:3]': 'SOC',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 419,
 'BOS': True,
 '+1:word.lower()': '-',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': ':',
 '+1:postag[:2]': ':',
 'w1': 4.7485266,
 'w2': 2.6400585,
 'w3': -0.33507606,
 'w4': 0.26797134,
 'w5': 1.3516893,
 'w6': -0.38730106,
 'w7': -1.9018749,
 'w8': -0.0078676315,
 'w9': 2.0295682,
 'w10': -1.5806023,
 'w11': -1.67731,
 'w12': -4.2850113,
 'w13': 2.2661965,
 'w14': -1.084038,
 'w15': -2.330188,
 'w16': -0.7616144,
 'w17': 3.92972,
 'w18': -3.7493434,
 'w19': 1.4172794,
 'w20': -6.079154,
 'w21': -2.8219287,
 'w22': -0.5769218,
 'w23': -3.387507,
 'w24': 0.99581206,
 'w25': 0.78039503,
 'w26': -1.5857658,
 'w27': 2.6818898,
 'w28': -1.1025083,
 'w29': -3.0312464,
 'w30': -0.43767452,
 'w31

In [16]:
X_sg[0][0]

{'word': 'SOCCER',
 'word.lower()': 'soccer',
 'word[-3:]': 'CER',
 'word[-2:]': 'ER',
 'word[:2]': 'SO',
 'word[:3]': 'SOC',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 419,
 'BOS': True,
 '+1:word.lower()': '-',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': ':',
 '+1:postag[:2]': ':',
 'w1': 1.4885135,
 'w2': 0.21551725,
 'w3': -0.30981103,
 'w4': 0.14827627,
 'w5': -0.837184,
 'w6': -0.28926045,
 'w7': -0.62283057,
 'w8': -0.90385467,
 'w9': 1.4134537,
 'w10': -0.040353056,
 'w11': -0.6016287,
 'w12': -0.2695964,
 'w13': 0.61027414,
 'w14': -0.36873552,
 'w15': -0.7872282,
 'w16': 0.23364611,
 'w17': 1.0941083,
 'w18': -0.6136633,
 'w19': 0.071141146,
 'w20': -1.7844874,
 'w21': -0.21196447,
 'w22': -0.7218415,
 'w23': 0.4215348,
 'w24': 0.6289073,
 'w25': -0.07872579,
 'w26': 0.8698475,
 'w27': -0.15904117,
 'w28': 0.6869201,
 'w29': -0.106255546,
 'w30': -0

In [17]:
X_fasttext[0][0]

{'word': 'SOCCER',
 'word.lower()': 'soccer',
 'word[-3:]': 'CER',
 'word[-2:]': 'ER',
 'word[:2]': 'SO',
 'word[:3]': 'SOC',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 419,
 'BOS': True,
 '+1:word.lower()': '-',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': ':',
 '+1:postag[:2]': ':',
 'w1': 0.78547615,
 'w2': 0.34772557,
 'w3': 1.0714978,
 'w4': 0.15954094,
 'w5': 0.4747368,
 'w6': -0.25446758,
 'w7': -1.2915962,
 'w8': -0.7018811,
 'w9': -0.32040668,
 'w10': 1.173651,
 'w11': 0.727166,
 'w12': 0.93836695,
 'w13': 0.06834475,
 'w14': 0.08571398,
 'w15': -0.34331954,
 'w16': -0.6032181,
 'w17': -0.74206614,
 'w18': -0.9422539,
 'w19': -0.5524901,
 'w20': 0.3896914,
 'w21': -1.1304176,
 'w22': 0.6276147,
 'w23': -0.05156191,
 'w24': -0.056948397,
 'w25': -0.34045145,
 'w26': -0.34159175,
 'w27': -0.085913666,
 'w28': -1.1645561,
 'w29': 0.35886627,
 'w30': 0.31

# Load Model

In [18]:
import pickle

In [20]:
filename_cbow = 'temp_crf_cbow_5.sav'
loaded_model_cbow = pickle.load(open(filename_cbow, 'rb'))

filename_sg = 'temp_crf_sg_5.sav'
loaded_model_sg = pickle.load(open(filename_sg, 'rb'))

filename_fasttext = 'temp_crf_fasttext_5.sav'
loaded_model_fasttext = pickle.load(open(filename_fasttext, 'rb'))

In [21]:
labels = list(loaded_model_cbow.classes_)
labels.remove('O')
labels

['B-ORG', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC']

In [22]:
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# Cross Val Score

In [23]:
x_val_score_cbow = cross_val_score(loaded_model_cbow, X_val_cbow, y_val, cv=5, scoring=f1_scorer)
print(x_val_score_cbow)
print(x_val_score_cbow.mean())

[0.80633198 0.6979645  0.83271769 0.81500635 0.68408608]
0.7672213230193291


In [24]:
x_val_score_sg = cross_val_score(loaded_model_sg, X_val_sg, y_val, cv=5, scoring=f1_scorer)
print(x_val_score_sg)
print(x_val_score_sg.mean())

[0.826746   0.67905911 0.82702053 0.81461881 0.6903014 ]
0.7675491706101054


In [25]:
x_val_score_fasttext = cross_val_score(loaded_model_fasttext, X_val_fasttext, y_val, cv=5, scoring=f1_scorer)
print(x_val_score_fasttext)
print(x_val_score_fasttext.mean())

[0.81538174 0.69728095 0.82006071 0.80813647 0.68634767]
0.7654415101612253


# CBOW

In [26]:
y_pred_cbow = loaded_model_cbow.predict(X_cbow)
metrics.flat_f1_score(y_test, y_pred_cbow,
                      average='weighted', labels=labels)

0.819186738971522

# SG

In [27]:
y_pred_sg = loaded_model_sg.predict(X_sg)
metrics.flat_f1_score(y_test, y_pred_sg,
                      average='weighted', labels=labels)

0.81531972626297

# FastText

In [28]:
y_pred_fasttext = loaded_model_fasttext.predict(X_fasttext)
metrics.flat_f1_score(y_test, y_pred_fasttext,
                      average='weighted', labels=labels)

0.8128669890644298